In [1]:
import re
import csv
obj = open("station_list.txt", "r")
station_list = [line.strip("\n") for line in obj]
obj.close()
print("station list obtained")

station list obtained


In [2]:
#get_data can retrieve the sounding data from a given station on a given day
#it strips the indices at the end of the sounding file; leaves a list of lists with full sounding column
def get_data(station,year,month,day):
	obj = open("./data/{}_{}_{}_{}.txt".format(str(station),str(year),str(month).zfill(2),str(day).zfill(2)), "r")
	data = []
	first_data = [line.strip("\n") for line in obj]		#strips new line character
	second_data = []
	for line in first_data:
		if line[re.search("\S",line).start()].isalpha() == False:	#gets rid of the bottom indeces
			second_data.append(line)
	try:
		second_data.pop(-1)
	except IndexError:
		data = []
        
	for line in second_data:
		new_line = [line[i:i+7] for i in range(0, len(line), 7)]	#splits it by columns, generating a list of lists
		new_new_line = [i.strip() for i in new_line]
		data.append(new_new_line)

	obj.close()
	return data
#get_CSV_data is the general csv data take in function, returns list of lists equivalent to the CSV
def get_CSV_data(file):
	csvFile = open(file, "r")
	csvReader = csv.reader(csvFile)
	csvGrid = list(csvReader)
	csvFile.close()
	return csvGrid
#get index gets the single value or (normal, virtual) tuple for a given index at a given date at a given station
#index values are taken from the end of the sounding file (they're precalculated by the Wyoming database)
#options for index input (as a string) are: cape, cin, bulk rich, lift, showaler, sweat, k, cross, vertical, totals
def get_index(index,station,year,month,day):
    try:
        obj = open("./data/{}_{}_{}_{}.txt".format(str(station),str(year),str(month).zfill(2),str(day).zfill(2)), "r")
        first_data = [line for line in obj]
        if index == "showalter" or index == "sweat" or index == "k" or index == "cross" or index == "vertical" or index == "totals":
            val = None
            for line in first_data:
                if index == "showalter":
                     if "Showalter index" in line:
                        val = float(line[44:])
                        break
                elif index == "sweat":
                    if "SWEAT index" in line:
                        val = float(line[44:])
                        break
                elif index == "k":
                    if "K index" in line:
                        val = float(line[44:])
                        break
                elif index == "cross":
                    if "Cross totals index" in line:
                        val = float(line[44:])
                        break
                elif index == "vertical":
                    if "Vertical totals index" in line:
                        val = float(line[44:])
                        break
                elif index == "totals":
                    if "Totals totals index" in line:
                        val = float(line[44:])
                        break
            return val
        elif index == "cape" or index == "cin" or index == "bulk rich" or index =="lift":
            val = None
            val_v = None
            for line in first_data:
                if index == "cape":
                    if "Convective Available Potential Energy" in line:
                        val = float(line[44:])
                    elif "CAPE using virtual temperature" in line:
                        val_v = float(line[44:])
                elif index == "cin":
                    if "Convective Inhibition" in line:
                        val = float(line[44:])
                    elif "CINS using virtual temperature" in line:
                        val_v = float(line[44:])
                elif index == "bulk rich":
                    if "Bulk Richardson Number" in line and "using CAPV" not in line:
                        val = float(line[44:])
                    elif "Bulk Richardson Number using CAPV" in line:
                        val_v = float(line[44:])
                elif index == "lift":
                    if "Lifted index" in line:
                        val = float(line[44:])
                    elif "LIFT computed using virtual temperature" in line:
                        val_v = float(line[44:])
            return (val, val_v)
        obj.close()
    except IOError:
        pass
    except ValueError:
        pass
#station_locations returns a dictionary of the form {station:(lat,long)}
#station latitude and longitude taken from the end of a sounidng file; provided by database
def station_locations(station_list):
	location_dic = {}
	for station in station_list:
		obj = open("./data/{}_2017_01_01.txt".format(station), "r")
		data = [line.strip("\n") for line in obj]
		for line in data:
			if "Station latitude" in line:		#find the latitude line and make a tuple with the latitude and the longitude(which is on the next line)
				location_dic[station] = (float(line[45:51]),float(data[data.index(line)+1][45:51]))
		obj.close()
	return location_dic
#county_locations returns a dictionary of the form {(county,ST):(lat,long)} (ST is the two-letter state abbreviation)
#where lat and long are the averaged values for all zipcodes in the county
def county_locations():
	zipcodes = get_CSV_data("./storm_data/zip_codes_states.csv")
	zipcodes.pop(0)
	county_zips = {}
	for code in zipcodes:
		try: 	#if it already exists, average the values
			if county_zips[(code[5],code[4])] and code[1] != '':
				county_zips[(code[5],code[4])] = ( (county_zips[(code[5],code[4])][0] + float(code[1]))/2 , (county_zips[(code[5],code[4])][1] + float(code[2]))/2 )
		except KeyError:
			if code[1] != '':
				county_zips[(code[5],code[4])] = (float(code[1]),float(code[2]))
	return county_zips
#storm_data returns a dictionary of the form {year:[CSV storm data for that year as obtained by get_CSV_data function]}
def storm_data():
	obj = open("./storm_data/file_list.txt", "r")
	file_list =  [line.strip("\n") for line in obj]
	data_dic = {}
	#populate the dictionary with years as keys and csv tables as values
	for i in file_list:
		data_dic[int(i[30:34])] = get_CSV_data("./storm_data/{}".format(i))
	obj.close()
	return data_dic
#state_abbrv converts the state name to its abbreviation (both as strings)
def state_abbrv(state):	#where state is a string
	state_list = get_CSV_data("./storm_data/states.csv")
	state_dic = {line[0]:line[1] for line in state_list}
	return state_dic[state]

In [ ]:
storm_dic = storm_data()	#year:CSV storm data for that year
county_loc_dic = county_locations()	#county:its location (tuple:tuple)
station_loc_dic = station_locations(station_list)	#station:its location

#station county dic has all the counties within acceptable distance of a station sorted by the stations to which they are relevant
station_county_dic = {} #format: {station:(station,[(county, ST),(other county, ST)])}
limit_lat = 0.43	#within one degree
limit_long = 0.56	#within one degree
#lets populate the station_county_dic:
for county in county_loc_dic:
    for station in station_loc_dic:
        if abs(float(county_loc_dic[county][0]) - float(station_loc_dic[station][0])) < limit_lat and abs(float(county_loc_dic[county][1]) - float(station_loc_dic[station][1])) < limit_long:
            try:
                station_county_dic[station][1].append(county)
            except KeyError:
                station_county_dic[station] = (station,[county])

In [ ]:
#index_events gets all the events associated with a particular index value/tuple for their day
#format: {event_id:(index tuple for that day) OR single value} including only events near one of 34 stations
#options for the index are the same as the options for get_index()
def index_events(index): 
    event_index_dic = {} 	
    for year in storm_dic.keys():
        for event in storm_dic[year][1:]:
            try:
                location = ()	#location should be a tuple
                info = [] #format: [station, year, month, day]
                if event[44] and event[45]:	#if this event has lat/long
                    for station in station_loc_dic:	#try each station and see if it's close enought to one
                        if abs(float(event[44]) - float(station_loc_dic[station][0])) < limit_lat and abs(float(event[45]) - float(station_loc_dic[station][1])) < limit_long:
                            location = (float(event[44]),float(event[45]))
                            info.append(station)
                            info.append(event[0][:4]) #year
                            info.append(event[0][4:]) #month
                            info.append(event[1].zfill(2))
                            event_index_dic[event[7]] = get_index(index,info[0],info[1],info[2],info[3])	#get the cape for this baby
                            break #how many does this break me out of? how does break work anyway?
                else:	#if it doesnt have lat/long, do the whole thing with county names
                    county = (event[15].title(),state_abbrv(event[8].title()))
                    if county in station_county_dic.values():
                        location = county_loc_dic[county]
                        info.append(station_county_dic[station][0])	#how do I get the station this is at????
                        info.append(event[0][:4]) #year
                        info.append(event[0][4:]) #month
                        info.append(event[1].zfill(2)) #day
                        event_index_dic[event[7]] = get_index(index,info[0],info[1],info[2],info[3])
            except KeyError:
                pass
            except ValueError:
                pass
    return event_index_dic

In [ ]:
#event_dates returns a dictionary with all the dates on which events took place
#format: {event id: (station,year,month,day)}
def event_dates(): 
    event_date_dic = {} 	
    for year in storm_dic.keys():
        for event in storm_dic[year][1:]: # event means line in csv
            try:
                location = ()	#location should be a tuple
                info = [] #format: [station, year, month, day]
                if event[44] and event[45]:	#if this event has lat/long
                    for station in station_loc_dic:	#try each station and see if it's close enought to one
                        if abs(float(event[44]) - float(station_loc_dic[station][0])) < limit_lat and abs(float(event[45]) - float(station_loc_dic[station][1])) < limit_long:
                            location = (float(event[44]),float(event[45]))
                            info.append(station)
                            info.append(event[0][:4]) #year
                            info.append(event[0][4:]) #month
                            info.append(event[1].zfill(2))
                            break #how many does this break me out of? how does break work anyway?
                else:	#if it doesnt have lat/long, do the whole thing with county names
                    county = (event[15].title(),state_abbrv(event[8].title()))
                    if county in station_county_dic.values():
                        location = county_loc_dic[county]
                        info.append(station_county_dic[station][0])	#how do I get the station this is at????
                        info.append(event[0][:4]) #year
                        info.append(event[0][4:]) #month
                        info.append(event[1].zfill(2)) #day
                if info != []:
                    event_date_dic[event[7]]=tuple(info)
            except KeyError:
                pass
            except ValueError:
                pass
    return event_date_dic

In [ ]:
#list of event ids that are tornados BUT not all are in the right geography (that is, includes some not near a station)
tornado_ids = []   
for year in storm_dic:
    data = storm_dic[year]
    for line in data:
        if "Tornado" in line[12]:
            event_id = line[7]
            tornado_ids.append(event_id)